In [2]:
import os
import numpy as np
import cv2
import preprocessing  
from feature_extraction import contruct_codebook, construct_bof_vector
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from joblib import dump, load  # For model persistence

In [4]:

classes_dic = {"IBM Plex Sans Arabic" : 0, "Lemonada" : 1, "Marhey" : 2, "Scheherazade New" : 3}
data_dir = "fonts-dataset"
raw_images = []
y = []
print("loading images ......")
for class_name in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_name)
    for filename in os.listdir(class_path):
        image_path = os.path.join(class_path, filename)
        image = preprocessing.preprocess(image_path, 3, 3) 
        raw_images.append(image)
        y.append(classes_dic[class_name])
raw_images = np.array(raw_images)
y = np.array(y)
print("Loading data is done ........")


loading images ......
Loading data is done ........


In [ ]:
print("constructing codebook ......")
codebook = contruct_codebook(raw_images)
print("codebook is done ........")
np.save('codebook.npy', codebook)

In [ ]:
x = np.zeros(len(y))

for i in range(len(raw_images)):
    x[i] = construct_bof_vector(raw_images[i], codebook)

print("Bof vector in x is done .........")

In [ ]:
# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)  # Set random_state for reproducibility


# Create and train the KNN model (experiment with different k values)
knn = KNeighborsClassifier(n_neighbors=5)  # You can adjust n_neighbors
knn.fit(X_train, y_train)

("training data is done ...........")
# Make predictions on the test set
y_pred = knn.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Optional: Save the trained model (using joblib)
model_path = "font_classification_model.pkl"  # Replace with your desired path
dump(knn, model_path)